# Digitstruct 파일 데이터 추출(to csv)

In [3]:
import h5py
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm
import pandas as pd

In [7]:
def get_box_data(data, index):
    meta_data = dict()
    meta_data['height'] = []
    meta_data['label'] = []
    meta_data['left'] = []
    meta_data['top'] = []
    meta_data['width'] = []

    def print_attrs(name, obj):
        vals = []
        if obj.shape[0] == 1:
            vals.append(obj[0][0])
        else:
            for k in range(obj.shape[0]):
                vals.append(int(data[obj[k][0]][0][0]))
        meta_data[name] = vals

    box = data['/digitStruct/bbox'][index]
    data[box[0]].visititems(print_attrs)
    return meta_data

In [8]:
mat_file = "./train2/digitStruct.mat"

In [9]:
def create_dateset(mat, dataset):
    data = h5py.File(mat, 'r')  # 경로
    df = []
    for i in tqdm(range(len(data['/digitStruct/name']))):
        meta_data = get_box_data(data, i)
        num_length = len(meta_data['label'])
        if num_length < 6:
            dd = {'filename': '%d.png'%(i+1), 'len': num_length}
            for i in range(5):
                dd['num%d'%(i+1)] = -1
                dd['bbox%d'%(i+1)] = (0, 0, 0, 0)
            for i in range(num_length):
                dd['num%d'%(i+1)] = int(meta_data['label'][i])
                dd['bbox%d'%(i+1)] = (meta_data['left'][i], meta_data['top'][i], meta_data['width'][i], meta_data['height'][i])
        df.append(dd)
    df = pd.DataFrame(df)
    for i in range(1, 6):
        df.loc[df[df['num%d'%i] == 10].index, 'num%d'%i] = 0
    for i in range(1, 6):
        df.loc[df['num%d'%i].isnull(), 'num%d'%i] = 10
    for i in range(1, 6):
        for j in df['bbox%d'%i][df['bbox%d'%i].isnull()].index:
            df.at[j, 'bbox%d'%(i+1)] = (0,0,0,0)

    df = df.dropna()
    df = df.reset_index(drop=True)

    df.to_csv('./%s.csv'%dataset)

In [10]:
create_dateset(mat_file, "train2")

100%|██████████| 13068/13068 [00:26<00:00, 492.93it/s]


# 추출한 csv 가공

In [12]:
df = pd.read_csv("./train2.csv")

list = []

for index, row in df.iterrows():
  for rpt in range(1, 6):
    if row[f"num{rpt}"] != -1:
      tmp_list = []
      tmp_list.append(row['filename'])
      bbox = row[f"bbox{rpt}"].strip("()").split(", ")

      tmp_list.append(int(float(bbox[3])))
      tmp_list.append(int(float(bbox[0])))
      tmp_list.append(int(float(bbox[1])))
      tmp_list.append(int(float(bbox[2])))

      tmp_list.append(row[f"num{rpt}"])
      list.append(tmp_list)

In [13]:
columns = ['filename','height','left','top','width' ,'label']

processed_df = pd.DataFrame(list , columns=columns)

In [15]:
processed_df.to_csv('./t2.csv')

# CSV파일과 이미지를 통해 이미지 데이터(32x32) 추출하기

In [ ]:
from PIL import Image
import pandas as pd
import numpy as np

data = pd.read_csv("t2.csv")
loc = "train2"  # 이미지 폴더 경로


def get_data(df, loc):

  x_data = []
  t_data = []

  for index, row in df.iterrows():
    # csv파일의 데이터들을 분류
    img = row["filename"]
    left = row["left"]
    upper = row["top"]
    right = left + row["width"]
    lower = upper + row["height"]
    number = row["label"]

    image = Image.open(f"{loc}/{img}")  # 이미지 불러오기
    num_img = image.crop((left, upper, right, lower))  # 이미지에서 숫자영역 크롭
    resize_img = num_img.resize((32, 32))  # 크롭된 이미지 리사이징(32, 32)
    img_data = np.array(resize_img)  # np array 픽셀 데이터로 변환

    x_data.append(img_data)
    t_data.append(number)

  x_data = np.array(x_data).astype('int64')
  t_data = np.array(t_data).astype('int64')

  return x_data, t_data

x_data, t_data = get_data(data,loc)

x_data.shape


In [17]:
np.save('x_data2.npy', x_data)
np.save('t_data2.npy', t_data)

------

In [5]:
x1 = np.load('x_data_all.npy')
x2 = np.load('x_data2.npy')

t1 = np.load('t_data_all.npy')
t2 = np.load('t_data2.npy')

In [6]:
x_con = np.concatenate((x1, x2), axis=0)
t_con = np.concatenate((t1, t2), axis=0)

In [7]:
print(x_con.shape)
print(t_con.shape)

(99286, 32, 32, 3)
(99286,)


In [8]:
np.save('x_concat.npy', x_con)
np.save('t_concat.npy', t_con)